L第12周作业：
1. 利用上周NER模型训练任务代码，复现课堂案例中：动态学习率、混合精度、DDP训练实现。
2. 利用课堂案例，实现分布式DDP模型训练。存盘后加载实现推理。

In [2]:
# 安装评估包
!pip -q install evaluate seqeval

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 6.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 193.6/193.6 kB 9.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cesium 0.12.4 requires numpy<3.0,>=2.0, but you have numpy 1.26.4 which is incompatible.
bigframes 1.42.0 requires rich<14,>=12.4.4, but you have rich 14.0.0 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.9.0.13 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cudnn-cu12==9.1.0.70; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cudnn-cu12 9.3.0.75 which is incompatible.
to

In [16]:
# 导包
from transformers import AutoModelForTokenClassification , AutoTokenizer 
from transformers import DataCollatorForTokenClassification , TrainingArguments , Trainer
from datasets import load_dataset
import numpy as np
import evaluate   # pip install evaluate
import seqeval  # pip install seqeval
from torch.utils.data import DataLoader
from torch.nn.utils.rnn import pad_sequence
from transformers import get_linear_schedule_with_warmup
import torch.optim as optim
import torch
from tqdm import tqdm

## 1、加载数据、加载分词器

In [6]:
# 加载数据
ds = load_dataset('doushabao4766/msra_ner_k_V3')

# 加载分词器
tokenizer = AutoTokenizer.from_pretrained('bert-base-chinese')

for items in ds['train']:
    print(items['tokens'])
    print(items['ner_tags'])
    break

README.md:   0%|          | 0.00/697 [00:00<?, ?B/s]

(…)-00000-of-00001-42717a92413393f9.parquet:   0%|          | 0.00/13.9M [00:00<?, ?B/s]

(…)-00000-of-00001-8899cab5fdab45bc.parquet:   0%|          | 0.00/946k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/45001 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3443 [00:00<?, ? examples/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/624 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/110k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/269k [00:00<?, ?B/s]

['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]


## 2、自定义实体映射字典 
’O':0   
'B-PER':1   
'I-PER':2   
'B-LOC':3   
'I-LOC':4   
'B-ORG':5   
'I-ORG':6 

In [8]:
# 查看tag标签数量
tags_id = set()
for tags in ds['train']:
    tags_id.update(tags['ner_tags'])

tags_id

{0, 1, 2, 3, 4, 5, 6}

In [9]:
# 构建映射标签
entites = list({'per' , 'loc' , 'org'})
tags = ['O']
for entity in entites:
    tags.append('B-' + entity.upper())  # upper()方法是转换为大写
    tags.append('I-' + entity.upper())
tags

['O', 'B-ORG', 'I-ORG', 'B-LOC', 'I-LOC', 'B-PER', 'I-PER']

In [10]:
# 创建构建方法 [tag + [0] * (512 - len(tag)) for tag in item['ner_tags']]
def data_input_proc(item):
    input_data = tokenizer(item['tokens'],
                          truncation = True ,  # 超过最大长度允许截断防止溢出
                          max_length = 512 ,   #最大512
                          add_special_tokens = False ,  # 禁止添加特殊标记  确保标签对其
                          is_split_into_words = True) # 因为该数据集已经按照字符划分，所以用id_split_into_words = True 表明一个字符一个字符的传入
    # 设置标签映射（超过512 截断）
    labels = [lbl[:512] for lbl in item['ner_tags']]
    input_data['labels'] = labels
    return input_data
ds1 = ds.map(data_input_proc , batched = True)


Map:   0%|          | 0/45001 [00:00<?, ? examples/s]

Map:   0%|          | 0/3443 [00:00<?, ? examples/s]

In [11]:
for item in ds1['train']:
    print(item['tokens'])
    print(item['ner_tags'])
    print(item['knowledge'])
    print(item['input_ids'])
    print(item['token_type_ids'])
    print(item['attention_mask'])
    print(item['labels'])
    break

['当', '希', '望', '工', '程', '救', '助', '的', '百', '万', '儿', '童', '成', '长', '起', '来', '，', '科', '教', '兴', '国', '蔚', '然', '成', '风', '时', '，', '今', '天', '有', '收', '藏', '价', '值', '的', '书', '你', '没', '买', '，', '明', '日', '就', '叫', '你', '悔', '不', '当', '初', '！']
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

[2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636, 674, 1036, 4997, 2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768, 7599, 3198, 8024, 791, 1921, 3300, 3119, 5966, 817, 966, 4638, 741, 872, 3766, 743, 8024, 3209, 3189, 2218, 1373, 872, 2637, 679, 2496, 1159, 8013]
[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
[0,

In [12]:
# 选择模型需要输入的列 将其转换为 torch张量类型
ds1.set_format('torch' , columns = ['input_ids' ,  # token 索引序列
                                    'token_type_ids' ,  # 段落标记
                                    'attention_mask' ,  # 注意力掩码
                                    'labels']) # NER标签序列
for item in ds1['train']:
    print(item)
    break

{'input_ids': tensor([2496, 2361, 3307, 2339, 4923, 3131, 1221, 4638, 4636,  674, 1036, 4997,
        2768, 7270, 6629, 3341, 8024, 4906, 3136, 1069, 1744, 5917, 4197, 2768,
        7599, 3198, 8024,  791, 1921, 3300, 3119, 5966,  817,  966, 4638,  741,
         872, 3766,  743, 8024, 3209, 3189, 2218, 1373,  872, 2637,  679, 2496,
        1159, 8013]), 'token_type_ids': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0]), 'attention_mask': tensor([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1]), 'labels': tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0])}


## 3 构建模型对象


In [13]:
# 构建模型初始化可读标签参数，
id2lbl = {i:tag for i,tag in enumerate(tags)}
lbl2id = {tag:i for i,tag in enumerate(tags)}

model = AutoModelForTokenClassification.from_pretrained("bert-base-chinese" , # 预训练模型
                                                       num_labels = len(tags) ,  # 输出的分类数量
                                                       id2label = id2lbl , 
                                                       label2id = lbl2id)
model

model.safetensors:   0%|          | 0.00/412M [00:00<?, ?B/s]

Some weights of BertForTokenClassification were not initialized from the model checkpoint at bert-base-chinese and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(21128, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

# 动态学习率

In [14]:
# 自动填充对其
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer , padding = True)
# 在DataLoader中使用
train_dl = DataLoader(
    ds1['train'], 
    batch_size=16,
    shuffle = True,
    collate_fn = data_collator
)


model.to('cuda')

# 模型参数分组

# 获取模型参数
param_optimizer = list(model.named_parameters())
bert_params, classifier_params = [],[]

for name,params in param_optimizer:
    # 获取预训练模型
    if 'bert' in name:
        bert_params.append(params)
    else:
        classifier_params.append(params)

param_groups = [
    {'params':bert_params, 'lr':1e-5},  # 预训练模型的学习率较低 保持稳定性
    {'params':classifier_params, 'weight_decay':0.1, 'lr':1e-3} # 新的分类层学习率较高 更好的学习，'weight_decay':0.1 使用正则化L2
]

# optimizer
optimizer = optim.AdamW(param_groups) # 优化器

# 学习率调度器

# 步长 从初始设置值到0 衰减需要的步长
train_steps = len(train_dl) * 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            # 预热 从0到初始设置值的步长
                                            num_warmup_steps=100, 
                                            # 衰减 从初始设置值到0 衰减需要的步长
                                            num_training_steps=train_steps)


for item in train_dl:
    print(item['input_ids'].shape, 
          item['token_type_ids'].shape, 
          item['attention_mask'].shape, 
          item['labels'].shape)
    break

DEVICE='cuda'

for epoch in range(5):
    model.train()
    tpbar = tqdm(train_dl)
    for items in tpbar:
        # 张量移动到指定的设备商
        items = {k:v.to(DEVICE) for k,v in items.items()}
        # 数据传入模型
        outputs = model(**items)
        # 计算损失
        loss = outputs.loss
        # 反向传播计算梯度
        loss.backward()
        # 更新模型参数的梯度
        optimizer.step()
        # 更新学习率
        scheduler.step()
        # 模型参数的梯度清零
        optimizer.zero_grad()
    
        tpbar.set_description(f'Epoch:{epoch+1} ' + 
                          f'bert_lr:{scheduler.get_lr()[0]} ' + 
                          f'classifier_lr:{scheduler.get_lr()[1]} '+
                          f'Loss:{loss.item():.4f}')


torch.Size([16, 140]) torch.Size([16, 140]) torch.Size([16, 140]) torch.Size([16, 140])


Epoch:1 bert_lr:8.057286072323668e-06 classifier_lr:0.0008057286072323666 Loss:0.0036: 100%|██████████| 2813/2813 [16:06<00:00,  2.91it/s]
Epoch:2 bert_lr:6.042964554242751e-06 classifier_lr:0.000604296455424275 Loss:0.0315: 100%|██████████| 2813/2813 [16:06<00:00,  2.91it/s]  
Epoch:3 bert_lr:4.028643036161834e-06 classifier_lr:0.0004028643036161833 Loss:0.0789: 100%|██████████| 2813/2813 [16:06<00:00,  2.91it/s]  
Epoch:4 bert_lr:2.014321518080917e-06 classifier_lr:0.00020143215180809166 Loss:0.0002: 100%|██████████| 2813/2813 [16:04<00:00,  2.92it/s] 
Epoch:5 bert_lr:0.0 classifier_lr:0.0 Loss:0.0003: 100%|██████████| 2813/2813 [16:03<00:00,  2.92it/s]                                      


# 混合精度

In [17]:
# 自动填充对其
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer , padding = True)
# 在DataLoader中使用
train_dl = DataLoader(
    ds1['train'], 
    batch_size=16,
    shuffle = True,
    collate_fn = data_collator
)


model.to('cuda')

# 模型参数分组

# 获取模型参数
param_optimizer = list(model.named_parameters())
bert_params, classifier_params = [],[]

for name,params in param_optimizer:
    # 获取预训练模型
    if 'bert' in name:
        bert_params.append(params)
    else:
        classifier_params.append(params)

param_groups = [
    {'params':bert_params, 'lr':1e-5},  # 预训练模型的学习率较低 保持稳定性
    {'params':classifier_params, 'weight_decay':0.1, 'lr':1e-3} # 新的分类层学习率较高 更好的学习，'weight_decay':0.1 使用正则化L2
]

# optimizer
optimizer = optim.AdamW(param_groups) # 优化器

# 学习率调度器

# 步长 从初始设置值到0 衰减需要的步长
train_steps = len(train_dl) * 5
scheduler = get_linear_schedule_with_warmup(optimizer, 
                                            # 预热 从0到初始设置值的步长
                                            num_warmup_steps=100, 
                                            # 衰减 从初始设置值到0 衰减需要的步长
                                            num_training_steps=train_steps)


# for item in train_dl:
#     print(item['input_ids'].shape, 
#           item['token_type_ids'].shape, 
#           item['attention_mask'].shape, 
#           item['labels'].shape)
#     break

DEVICE='cuda'

# 梯度计算缩放器
scaler = torch.GradScaler()

for epoch in range(3):
    model.train()
    tpbar = tqdm(train_dl)
    for items in tpbar:
        # 张量移动到指定的设备商
        items = {k:v.to(DEVICE) for k,v in items.items()}
        # 数据传入模型
        outputs = model(**items)
        # 计算损失
        loss = outputs.loss
        
        # 缩放loss后 调用反向传播计算梯度
        scaler.scale(loss).backward()
        # 更新模型参数的梯度
        scaler.step(optimizer)
        scaler.update()

        
        # 更新学习率
        scheduler.step()
        # 模型参数的梯度清零
        optimizer.zero_grad()
    
        tpbar.set_description(f'Epoch:{epoch+1} ' + 
                          f'bert_lr:{scheduler.get_lr()[0]} ' + 
                          f'classifier_lr:{scheduler.get_lr()[1]} '+
                          f'Loss:{loss.item():.4f}')

Epoch:1 bert_lr:8.057286072323668e-06 classifier_lr:0.0008057286072323666 Loss:0.0150: 100%|██████████| 2813/2813 [16:17<00:00,  2.88it/s]
Epoch:2 bert_lr:6.042964554242751e-06 classifier_lr:0.000604296455424275 Loss:0.0001: 100%|██████████| 2813/2813 [16:19<00:00,  2.87it/s]  
Epoch:3 bert_lr:4.028643036161834e-06 classifier_lr:0.0004028643036161833 Loss:0.0013: 100%|██████████| 2813/2813 [16:19<00:00,  2.87it/s]  


## 4 创建评估函数

In [10]:

def compute_metric(result):
    # 传入的result是一个元祖 (predicts,labels)

    # 加载序列标注评估指标库
    seqeval = evaluate.load('seqeval')
    # 解构模型输出的结果
    predicts,labels = result
    # 沿着axis = 2 的维度 取最大值索引 然后将predicts转换为预测标签ID
    predicts = np.argmax(predicts , axis = 2)
    # 准备评估数据 将数字ID转换为文本标签 并且过滤填充数值-100
    predicts = [[tags[p] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    labels = [[tags[l] for p,l in zip(ps,ls) if l != -100]
                 for ps,ls in zip(predicts,labels)]
    results = seqeval.compute(predictions = predicts , references = labels)
    return results

[[[0.1, 0.8, 0.1],  # 预测为1(B-PER)
  [0.3, 0.2, 0.5],  # 预测为2(I-PER) 
  [0.9, 0.05, 0.05]] # 预测为0(O)
]
经过argsmax转换后得到
[1, 2, 0]

## 5 批量处理序列数据 动态填充长度保证对齐

In [63]:
data_collator = DataCollatorForTokenClassification(tokenizer = tokenizer , padding = True)

## 6 设置模型训练相关参数 TrainingArguments

In [12]:
args = TrainingArguments(
    output_dir = 'ner_train' , # 设置模型输出目录
    num_train_epochs = 3 , # 训练轮数
    #save_safetensor = False # 模型禁止保存safe格式 可以用troch.load加载
    per_device_train_batch_size = 32 , # 训练批次
    per_device_eval_batch_size = 32 ,  # 评估批次
    report_to = 'tensorboard' , # 设置训练输出记录为tensorboard
    eval_strategy = 'epoch'  # 每轮评估一次
)


## 7 创建模型训练

In [13]:
trainer = Trainer(
    model = model ,  # 指定模型
    args = args , # 指定设置参数
    train_dataset = ds1['train'] ,  # 输入训练数据
    eval_dataset = ds1['test'] ,  # 输入评估数据
    compute_metrics = compute_metric , # 指定评估函数
    data_collator = data_collator  # 指定数据收集器
)

## 8 模型训练

In [14]:
trainer.train()

/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Epoch,Training Loss,Validation Loss,Loc,Org,Per,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
1,0.061700,0.026402,"{'precision': 0.9412915851272016, 'recall': 0.9600798403193613, 'f1': 0.950592885375494, 'number': 1503}","{'precision': 0.8319559228650137, 'recall': 0.9151515151515152, 'f1': 0.8715728715728714, 'number': 1320}","{'precision': 0.9541054141269272, 'recall': 0.9330294530154277, 'f1': 0.9434497429533769, 'number': 2852}",0.919986,0.936035,0.927941,0.992429
2,0.019900,0.028749,"{'precision': 0.9419439008480104, 'recall': 0.9607451763140386, 'f1': 0.9512516469038209, 'number': 1503}","{'precision': 0.8693790149892934, 'recall': 0.9227272727272727, 'f1': 0.8952590959206175, 'number': 1320}","{'precision': 0.952567760342368, 'recall': 0.9365357643758766, 'f1': 0.9444837340876945, 'number': 2852}",0.929418,0.939736,0.934548,0.992964
3,0.006600,0.031673,"{'precision': 0.9476439790575916, 'recall': 0.9634065202927479, 'f1': 0.9554602441438469, 'number': 1503}","{'precision': 0.8745493871665465, 'recall': 0.918939393939394, 'f1': 0.8961950498707056, 'number': 1320}","{'precision': 0.954561590344338, 'recall': 0.9428471248246845, 'f1': 0.9486681954489328, 'number': 2852}",0.933357,0.942731,0.938021,0.993275


/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.
  warnings.warn(


Trainer is attempting to log a value of "{'precision': 0.9412915851272016, 'recall': 0.9600798403193613, 'f1': 0.950592885375494, 'number': 1503}" of type <class 'dict'> for key "eval/LOC" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.8319559228650137, 'recall': 0.9151515151515152, 'f1': 0.8715728715728714, 'number': 1320}" of type <class 'dict'> for key "eval/ORG" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
Trainer is attempting to log a value of "{'precision': 0.9541054141269272, 'recall': 0.9330294530154277, 'f1': 0.9434497429533769, 'number': 2852}" of type <class 'dict'> for key "eval/PER" as a scalar. This invocation of Tensorboard's writer.add_scalar() is incorrect so we dropped this attribute.
/usr/local/lib/python3.11/dist-packages/torch/nn/parallel/_functions.py:70: UserWarning: Was asked to ga

TrainOutput(global_step=2112, training_loss=0.02409702581776814, metrics={'train_runtime': 2452.6864, 'train_samples_per_second': 55.043, 'train_steps_per_second': 0.861, 'total_flos': 1.180990200098808e+16, 'train_loss': 0.02409702581776814, 'epoch': 3.0})

## 9 输入语句检测

In [18]:
from transformers import pipeline

pipeline = pipeline('token-classification', 'ner_train/checkpoint-2112')

text = pipeline('双方确定了今后发展中美关系的指导方针')


Device set to use cuda:0
